In [ ]:
import numpy as np
import pandas as pd
from  matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.svm import SVC
from scipy import sparse

# Load data

In [ ]:
train_df = pd.read_csv("/content/drive/My Drive/vlsp/clean_train_df_ext.csv")
test_df = pd.read_csv("/content/drive/My Drive/vlsp/clean_final_test_df.csv")

In [ ]:
train_df[:2586]

,id,user_name,timestamp_post,num_like_post,num_comment_post,num_share_post,label,clean_post_message
0,0,2.16707472383313e+18,1.584426e+09,45.0,15.0,8.0,1,"Cần các bậc phụ_huynh xã Ngũ_Thái lên_tiếng , ..."
1,1,7.36849754781241e+18,1.581363e+09,979.0,39.0,138.0,1,KÊU_GỌI ĂN_CHAY CẦU_NGUYỆN XIN CHÚA CỨU KHỎI D...
2,2,-5.75414262528031e+18,1.587042e+09,85.0,13.0,61.0,1,"Giàn khoan dầu_khí gặp sự_cố , chết người ! Gi..."
3,3,4.17793530884965e+18,1.596449e+09,114.0,12.0,5.0,1,"Thuận_Lợi có ca dương_tính CV19 rồi đó , mọi n..."
4,4,5.540309800746e+18,1.583548e+09,166.0,4.0,21.0,1,Sa_Pa cho 9 người khách nước_ngoài đi cùng chu...
...,...,...,...,...,...,...,...,...
2581,1782,808e278b22ec6b96f2faf7447d10cd8e,1.584934e+09,0.0,0.0,0.0,0,Thông_báo cho người cách_ly cần tuân_thủ quy_đ...
2582,1783,55edeb1818594cdccb91fbf2c5cbed8b,1.584230e+09,4100.0,1400.0,2000.0,0,"Tối 14-3 , Giám_đốc Sở Y_tế Phan_Huy_Anh_Vũ ch..."
2583,1784,808e278b22ec6b96f2faf7447d10cd8e,1.587168e+09,7.0,0.0,0.0,0,"Trưa 18/4 , Bộ Y_tế công_bố khỏi bệnh thêm 3 t..."
2584,1785,7284a5b886dcf89f77411470ffe9bd61,1.585892e+09,6007.0,50.0,1136.0,0,Hướng_dẫn việc thực_hiện cách_ly xã_hội ; danh...


In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5172 entries, 0 to 5171
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  5172 non-null   int64  
 1   user_name           5172 non-null   object 
 2   timestamp_post      5172 non-null   float64
 3   num_like_post       5172 non-null   float64
 4   num_comment_post    5172 non-null   float64
 5   num_share_post      5172 non-null   float64
 6   label               5172 non-null   int64  
 7   clean_post_message  5102 non-null   object 
dtypes: float64(4), int64(2), object(2)
memory usage: 323.4+ KB


In [ ]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1646 entries, 0 to 1645
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  1646 non-null   int64  
 1   user_name           1646 non-null   object 
 2   timestamp_post      1646 non-null   float64
 3   num_like_post       1646 non-null   float64
 4   num_comment_post    1646 non-null   float64
 5   num_share_post      1646 non-null   float64
 6   len_post_message    1646 non-null   int64  
 7   clean_post_message  1616 non-null   object 
dtypes: float64(4), int64(2), object(2)
memory usage: 103.0+ KB


In [ ]:
train_df['clean_post_message'] = train_df['clean_post_message'].astype(str)
test_df['clean_post_message'] = test_df['clean_post_message'].astype(str)

In [ ]:
y = train_df['label'].values
y

array([1, 1, 1, ..., 1, 1, 0])

In [ ]:
train_df.shape, test_df.shape

((5172, 8), (1646, 8))

In [ ]:
# vec = TfidfVectorizer(ngram_range=(1, 2))

# CV for choosing ngram_range param

In [ ]:
kf = KFold(n_splits=5)
y_pred = np.zeros((1646, 2))
for (i, (train_idx, val_idx)) in enumerate(kf.split(train_df)):
  X_train, X_val = train_df.loc[train_idx], train_df.loc[val_idx]
  y_train, y_val = y[train_idx], y[val_idx]

  # print(X_train)
  vec = TfidfVectorizer(ngram_range=(2, 4), analyzer="char")
  X_train_vec = vec.fit_transform(X_train['clean_post_message'])
  X_val_vec = vec.transform(X_val['clean_post_message'])
  # r = sparse.csr_matrix(np.log(pr(X_train_vec, 1, y_train), pr(X_train_vec, 0, y_train)))
  
  m = LogisticRegression(dual=True, solver='liblinear')
  X_train_nb = X_train_vec
  m.fit(X_train_nb, y_train)

  X_test_vec = vec.transform(test_df['clean_post_message'])
  X_test_nb = X_test_vec
  y_scores = m.predict_proba(X_val_vec)
  y_pred = np.sum([m.predict_proba(X_test_nb), y_pred], axis=0)
  print("Fold", i, ":", roc_auc_score(y_val, y_scores[:, 1]))


Fold 0 : 0.9173846499630752
Fold 1 : 0.9180865700074081
Fold 2 : 0.9177390172999125
Fold 3 : 0.8974306422462783
Fold 4 : 0.9098067942496109


In [ ]:
y_pred

array([[3.09538339, 1.90461661],
       [3.87556245, 1.12443755],
       [4.36444478, 0.63555522],
       ...,
       [4.5842046 , 0.4157954 ],
       [4.64859298, 0.35140702],
       [4.2358901 , 0.7641099 ]])

In [ ]:
y_pred_re = y_pred[:, 1]/5

In [ ]:
y_pred_re

array([0.38092332, 0.22488751, 0.12711104, ..., 0.08315908, 0.0702814 ,
       0.15282198])

In [ ]:
result = pd.DataFrame()
result['id'] = test_df['id']
result['label'] = y_pred_re

In [ ]:
result.to_csv("results.csv", index=False, header=None)

# Stacking part

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_strain, X_sval, y_strain, y_sval = train_test_split(train_df, y, test_size=0.3, shuffle=False)

In [ ]:
X_strain

,id,user_name,timestamp_post,num_like_post,num_comment_post,num_share_post,label,clean_post_message
0,0,2.16707472383313e+18,1.584426e+09,45.0,15.0,8.0,1,"Cần các bậc phụ_huynh xã Ngũ_Thái lên_tiếng , ..."
1,1,7.36849754781241e+18,1.581363e+09,979.0,39.0,138.0,1,KÊU_GỌI ĂN_CHAY CẦU_NGUYỆN XIN CHÚA CỨU KHỎI D...
2,2,-5.75414262528031e+18,1.587042e+09,85.0,13.0,61.0,1,"Giàn khoan dầu_khí gặp sự_cố , chết người ! Gi..."
3,3,4.17793530884965e+18,1.596449e+09,114.0,12.0,5.0,1,"Thuận_Lợi có ca dương_tính CV19 rồi đó , mọi n..."
4,4,5.540309800746e+18,1.583548e+09,166.0,4.0,21.0,1,Sa_Pa cho 9 người khách nước_ngoài đi cùng chu...
...,...,...,...,...,...,...,...,...
3615,2816,e3b61c45d5b844d85872dcad37dd514d,1.584234e+09,873.0,16.0,81.0,0,"Vào rạng sáng 14/3 , một vụ đổ trộm chất_thải ..."
3616,2817,389c669730cb6c54314a46be785cea42,1.583515e+09,33000.0,3400.0,9500.0,0,THÔNG_TIN CHI_TIẾT VỀ CA BỆNH THỨ 17 MẮC COVID...
3617,2818,7acdbd2e623bc27e96d0f4fc87b15ae5,1.588008e+09,26.0,1.0,-999.0,0,"Hoãn phiên_toà phúc_thẩm xét_xử Tiến "" trắng ""..."
3618,2819,acc434a9db7457c22f17ccb42fba178b,1.580912e+09,377.0,119.0,1900.0,1,TRÒ LỪA_ĐẢO QUÁ TÀN_NHẪN Liên_quan đến virus c...


In [ ]:
vec = TfidfVectorizer(ngram_range=(2, 4), analyzer="char")
X_strain_vec = vec.fit_transform(X_strain['clean_post_message'])
X_sval_vec = vec.transform(X_sval['clean_post_message'])

In [ ]:
m = LogisticRegression(dual=True, solver='liblinear')
m.fit(X_strain_vec, y_strain)

X_stest_vec = vec.transform(test_df['clean_post_message'])

In [ ]:
y_sval_pred = m.predict_proba(X_sval_vec)[:, 1]
y_test_pred = m.predict_proba(X_stest_vec)[:, 1]

In [ ]:
pd.DataFrame(y_sval_pred, columns=["y_sval_pred_lr"]).to_csv('y_sval_pred_lr.csv', index=False)
pd.DataFrame(y_test_pred, columns=["y_test_pred_lr"]).to_csv('y_test_pred_lr.csv', index=False)

In [ ]:
y_sval

array([0, 0, 0, ..., 1, 1, 0])